In [ ]:
using Plots
using Random


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [ ]:
sigmoid(x) = 1.0 ./ (1.0 .+ exp.(-x))
sigmoid_derivative(x) = #Implement derivative of sigmoid function here

In [ ]:
mutable struct NeuralNetwork
    inputSize::Int
    hiddenSize::Int
    outputSize::Int
    
    weightsLayerOne::Matrix{Float64}
    weightsLayerTwo::Matrix{Float64}
    
    learningRate::Float64
    optimiser::Symbol
 
    function NeuralNetwork(inputSize::Int, hiddenSize::Int, outputSize::Int; 
                          learningRate::Float64=0.1,
                          optimiser::Symbol=:gd)
        
        weightsLayerOne = randn(hiddenSize, inputSize) 
        weightsLayerTwo = randn(outputSize, hiddenSize) 
        
        new(inputSize, hiddenSize, outputSize, 
            weightsLayerOne, weightsLayerTwo,
            learningRate, optimiser)
    end
end

In [ ]:
function forward(self::NeuralNetwork, X::Matrix{Float64})

    outputLayerOne = (self.weightsLayerOne * X)
    activationLayerOne = sigmoid(outputLayerOne)

    outputLayerTwo = (self.weightsLayerTwo * activationLayerOne) 
    activationLayerTwo = sigmoid(outputLayerTwo)
    
    return outputLayerOne, activationLayerOne, outputLayerTwo, activationLayerTwo
end

In [ ]:
function predict(self::NeuralNetwork, X::Matrix{Float64})
    _, _, _, activationLayerTwo = forward(self, X)
    return activationLayerTwo
end

In [ ]:
# MSE loss
function calculate_loss(self::NeuralNetwork, X::Matrix{Float64}, y::Matrix{Float64})
    y_pred = predict(self, X)
    return #Implement MSE loss function here
end

In [ ]:
function train!(self::NeuralNetwork, X::Matrix{Float64}, y::Matrix{Float64}, epochs::Int=15000)
    losses = []
    
    for epoch in 1:epochs
        
        # Forward pass
        outputLayerOne, activationLayerOne, outputLayerTwo, activationLayerTwo = forward(self, X)
        
        # Calculate loss
        loss = calculate_loss(self, X, y)
        push!(losses, loss)
        
        # Implement backward pass based on partial derivative
        

        # Implement gradient descent, momentum, Adagrad and Adam
        if self.optimiser == :gd
            # Implement gradient descent here
        elseif self.optimiser == :momentum
            # Implement momentum here
        elseif self.optimiser == :adagrad
            # Implement adagrad here
        elseif self.optimiser == :adam
            # Implement Adam here
        end
            
        if epoch % 1000 == 0
            println("Epoch $epoch, Loss: $loss")
        end
    end
    
    return losses
end

# XOR function input and output
X = [[0.0,0.0]  [0.0,1.0]  [1.0,0.0] [1.0, 1.0]]
y = [0.0 1.0 1.0 0.0]

nn = NeuralNetwork(2, 5, 1, learningRate=0.1, optimiser=:gd)
#nn = NeuralNetwork(2, 5, 1, learningRate=0.1, optimiser=:momentum)
#nn = NeuralNetwork(2, 5, 1, learningRate=0.1, optimiser=:adagrad)
#nn = NeuralNetwork(2, 5, 1, learningRate=0.1, optimiser=:adam)

# Test the untrained model
println("\nResults before training:")
for i in 1:size(X, 2)
    input = X[:, i:i] 
    prediction = predict(nn, input)
    println("Input: [$(X[1,i]), $(X[2,i])], Predicted: $(round(prediction[1], digits=5)), Expected: $(y[i])")
end

# Train the neural network and plot the loss
losses = train!(nn, X, y, 15000)
p = plot(1:length(losses), losses, 
         xlabel="Epoch", 
         ylabel="Loss (MSE)", 
         title="Neural Network Training Loss for XOR", 
         legend=false, 
         lw=2,
         yscale=:log10)
display(p)

# Test the trained model
println("\nResults after training:")
for i in 1:size(X, 2)
    input = X[:, i:i]
    prediction = predict(nn, input)
    println("Input: [$(X[1,i]), $(X[2,i])], Predicted: $(round(prediction[1], digits=5)), Expected: $(y[i])")
end